# Data Encoding

This notebook contains the code related to the data-encoding project. More details are in the `README.md`. Much of the below language is taken directly from the authors of the paper given in the `README.md`.

## Notation

$D=(x^d, y^d)_{d=1}^m$ represents a classical data set, where $m$ is the number of samples, $x^d \in \mathbb{R}^N$ is an $N$-dimensional input data point (sample) and $y^d$ is the label for the corresponding sample.

For the purposes of this notebook, we will encode the following 4-dimensional data point. 

$$x = [0.1, 0.3, 0.2, -0.5]$$

In [7]:
x = [0.1, 0.3, 0.2, -0.5]

In [15]:
import pennylane as qml
import numpy as np

## Angle encoding

### Simple Angle Encoding

For a data point (sample) $x^d$, the angle encoding method uses rotation gates ($R_x$ and $R_y$) whose angles correspond to the fields of $x^d$.

$x^d$ is encoded into $$\ket{\psi^d}=\bigotimes_{j=1}^{N} \ket{\psi_j^d}$$

where $\ket{\psi_j}$ is given by $$\ket{\psi_j} = R_x(x_j)\ket{0} = \cos \frac{x_j}{2} \ket{0} - i \sin \frac{x_j}{2} \ket{1}$$ or 

$$\ket{\psi_j} = R_y(x_j)\ket{0} = \cos \frac{x_j}{2} \ket{0} + \sin \frac{x_j}{2} \ket{1}$$

depending on if $R_x$ or $R_y$ is used.





So for each $x_j$ we apply a rotation gate onto a qubit. Thus we need 4-qubits to encode this state.

In [2]:
dev = qml.device("default.qubit", wires=4)

In [ ]:
@qml.qnode(dev)
def simple_angle_encoding(x, rotator="RX"):
    op = qml.RY if rotator == "RY" else qml.RX
    for i in range(len(x)):
        op(x[i], wires=i)
    
    return qml.state()

Testing the Simple Angle Encoding function with $R_Y$:

In [ ]:
print(qml.draw(simple_angle_encoding)(x, rotator="RY"))

0: ──RY(0.10)──┤  State
1: ──RY(0.30)──┤  State
2: ──RY(0.20)──┤  State
3: ──RY(-0.50)─┤  State


In [ ]:
simple_angle_encoding(x, rotator="RY")

array([ 9.52055097e-01+0.j, -2.43099578e-01+0.j,  9.55241360e-02+0.j,
       -2.43913164e-02+0.j,  1.43889055e-01+0.j, -3.67409077e-02+0.j,
        1.44370611e-02+0.j, -3.68638692e-03+0.j,  4.76424635e-02+0.j,
       -1.21651182e-02+0.j,  4.78019096e-03+0.j, -1.22058314e-03+0.j,
        7.20045412e-03+0.j, -1.83857779e-03+0.j,  7.22455203e-04+0.j,
       -1.84473099e-04+0.j])

Now with $R_X$

In [ ]:
print(qml.draw(simple_angle_encoding)(x, rotator="RX"))

0: ──RX(0.10)──┤  State
1: ──RX(0.30)──┤  State
2: ──RX(0.20)──┤  State
3: ──RX(-0.50)─┤  State


In [ ]:
simple_angle_encoding(x, rotator="RX")

array([ 9.52055097e-01+0.j        ,  0.00000000e+00+0.24309958j,
        0.00000000e+00-0.09552414j,  2.43913164e-02+0.j        ,
        0.00000000e+00-0.14388905j,  3.67409077e-02+0.j        ,
       -1.44370611e-02+0.j        ,  0.00000000e+00-0.00368639j,
        0.00000000e+00-0.04764246j,  1.21651182e-02+0.j        ,
       -4.78019096e-03+0.j        ,  0.00000000e+00-0.00122058j,
       -7.20045412e-03+0.j        ,  0.00000000e+00-0.00183858j,
        0.00000000e+00+0.00072246j, -1.84473099e-04+0.j        ])

### $\frac{\pi}{4}$-Angle encoding

This encoding method uses the Walsh-Hadamard operator and a unitary given by the following

$$U(x) = \begin{pmatrix}
\cos (\frac{\pi}{4} - x) & \sin(\frac{\pi}{4} - x) \\ 
-\sin(\frac{\pi}{4} - x) & \cos(\frac{\pi}{4} - x)


\end{pmatrix}$$

where each qubit $\ket{\psi_j}=U(x_j)H\ket{0}$. The result of this is given by $\ket{\psi_j} = \cos x_j \ket{0} + \sin x_j \ket{1}$, in which the input to the trigonometric functions are halved in contrast to the Simple Angle Encoding method that uses $R_Y$.

In [19]:
def _construct_Uj(xj):
    return [
            [np.cos(np.pi/4 - xj), np.sin(np.pi/4 - xj)],
            [-np.sin(np.pi/4 - xj), np.cos(np.pi/4 - xj)]
        ]

@qml.qnode(dev)
def pi_4_angle_encoding(x):
    for i in range(len(x)):
        qml.Hadamard(i)
        qml.QubitUnitary(_construct_Uj(x[i]), wires=i)

    return qml.state()

In [22]:
print(qml.draw(pi_4_angle_encoding)(x))

0: ──H──U(M0)─┤  State
1: ──H──U(M1)─┤  State
2: ──H──U(M2)─┤  State
3: ──H──U(M3)─┤  State

M0 = 
[[ 0.77416708  0.63298131]
 [-0.63298131  0.77416708]]
M1 = 
[[ 0.88448925  0.46656057]
 [-0.46656057  0.88448925]]
M2 = 
[[ 0.83349215  0.55253129]
 [-0.55253129  0.83349215]]
M3 = 
[[ 0.28153953  0.95954963]
 [-0.95954963  0.28153953]]


In [20]:
pi_4_angle_encoding(x)

array([ 0.81756978+0.j, -0.44664041+0.j,  0.1657296 +0.j, -0.09053849+0.j,
        0.25290397+0.j, -0.13816207+0.j,  0.05126617+0.j, -0.02800684+0.j,
        0.0820306 +0.j, -0.04481352+0.j,  0.01662842+0.j, -0.00908415+0.j,
        0.02537504+0.j, -0.01386245+0.j,  0.00514377+0.j, -0.00281006+0.j])

### Entangled Angle Encoding

Entangled Angle Encoding builds on Simple Angle Encoding by adding CNOT between each pair of adjacent qubits.

In [30]:
def _simple_angle_encoding_no_return(x, rotator="RX"):
    op = qml.RY if rotator == "RY" else qml.RX
    for i in range(len(x)):
        op(x[i], wires=i)



@qml.qnode(dev)
def entangled_angle_encoding(x, rotator="RX"):
    _simple_angle_encoding_no_return(x, rotator)
    for i in range(len(x) - 1):
        qml.CNOT(wires=[i, i + 1])
    
    qml.CNOT(wires=[len(x) - 1, 0])

    return qml.state()

In [31]:
print(qml.draw(entangled_angle_encoding)(x))

0: ──RX(0.10)──╭●───────╭X─┤  State
1: ──RX(0.30)──╰X─╭●────│──┤  State
2: ──RX(0.20)─────╰X─╭●─│──┤  State
3: ──RX(-0.50)───────╰X─╰●─┤  State


In [32]:
entangled_angle_encoding(x)

array([ 9.52055097e-01+0.j        ,  0.00000000e+00-0.00183858j,
        2.43913164e-02+0.j        ,  0.00000000e+00+0.00072246j,
       -1.44370611e-02+0.j        ,  0.00000000e+00-0.00122058j,
        3.67409077e-02+0.j        ,  0.00000000e+00-0.04764246j,
       -7.20045412e-03+0.j        ,  0.00000000e+00+0.24309958j,
       -1.84473099e-04+0.j        ,  0.00000000e+00-0.09552414j,
       -4.78019096e-03+0.j        ,  0.00000000e+00-0.00368639j,
        1.21651182e-02+0.j        ,  0.00000000e+00-0.14388905j])

In [33]:
print(qml.draw(entangled_angle_encoding)(x, rotator="RY"))

0: ──RY(0.10)──╭●───────╭X─┤  State
1: ──RY(0.30)──╰X─╭●────│──┤  State
2: ──RY(0.20)─────╰X─╭●─│──┤  State
3: ──RY(-0.50)───────╰X─╰●─┤  State


In [34]:
entangled_angle_encoding(x, rotator="RY")

array([ 9.52055097e-01+0.j, -1.83857779e-03+0.j, -2.43913164e-02+0.j,
        7.22455203e-04+0.j,  1.44370611e-02+0.j, -1.22058314e-03+0.j,
       -3.67409077e-02+0.j,  4.76424635e-02+0.j,  7.20045412e-03+0.j,
       -2.43099578e-01+0.j, -1.84473099e-04+0.j,  9.55241360e-02+0.j,
        4.78019096e-03+0.j, -3.68638692e-03+0.j, -1.21651182e-02+0.j,
        1.43889055e-01+0.j])